<a href="https://colab.research.google.com/github/DreamTeamInc/Core-DS/blob/issues%2F2/Kern_Test_Classification_Leon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Команда ниже монтирует образ gDrive. Добавь ярлык на папку с керном перед этим.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Path to 'sample_core_analysis_data_example' folder with folder name
SAMPLE_CORE_PATH = 'drive/My Drive/ sample_core_analysis_data_example'

Mounted at /content/drive


# Инициализация библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import copy

from skimage import io

from IPython.display import clear_output
pd.options.mode.chained_assignment = None
np.random.seed(0)

# Подготовка данных

In [3]:
data = pd.read_csv(SAMPLE_CORE_PATH + '/data.csv')
print(data.shape)
data.sample(3)

(14915, 6)


,task_id,photo_id,photo_type,segment_num,segment_type,segment_value
1078,3a5eb82d_6722_48e2_b7e6_311e26059040,1004664,ДС,6,Разрушенность,Разлом
3865,b62eb3e8_1b20_449a_99b7_9177f3dc3adb,1003306,ДС,4,Разрушенность,Разлом
10183,575dc208_d70e_42f1_a1fc_540d04bf551e,1009500,ДС,1,Разрушенность,Разлом


Выбираем на фото дневные фотографии с сегментами типа 'Порода' без 'Переслаивания пород'

In [4]:
data_day = data[data['photo_type'] == 'ДС'] 
data_day_rock = data_day[data_day['segment_type'] == 'Порода']
data_day_rock_light = data_day_rock[data_day_rock['segment_value'] != 'Переслаивание пород'] 
print(data_day_rock_light.shape)
data_day_rock_light.sample(3)

(1856, 6)


,task_id,photo_id,photo_type,segment_num,segment_type,segment_value
10884,d2e9e4b9_ef9c_4d1c_9399_7ab13a1c8de3,1005436,ДС,12,Порода,Песчаник
1412,f05f4cbb_f3f3_4983_90ad_38da9f5ceb9d,1016562,ДС,0,Порода,Аргиллит
2195,f4c6385c_4bd2_4b7e_8c5b_7b79ddbfefb4,1005036,ДС,7,Порода,Алевролит глинистый


Вспомогатаельная функция для взятия **n** **task**'ов для каждого типа породы. Параметр **dropout** определяет стоит ли выкидывать классы сегментов, которые имеют меньше **n** **task**'ов

In [5]:
def take_n_samples(data, n, dropout=True):
    df = pd.DataFrame(data=None, columns=data.columns)
    rocks = pd.unique(data[data['segment_type'] == 'Порода']['segment_value'].values)

    for rock in rocks:
        data_rock = data[data["segment_value"] == rock]
        if dropout and len(data_rock) < n:
            continue
        for i in range(min(n, len(data_rock))):
            df = df.append(data_rock.iloc[i])
        
    return df

Берём по 50 примеров из data_day_rock_light, выкидывая классы с меньшим количеством **task_id**

In [6]:
data_xy = take_n_samples(data_day_rock_light, 50)
data_xy = data_xy.sample(200)
print(data_xy.shape)
data_xy.sample(3)

(200, 6)


,task_id,photo_id,photo_type,segment_num,segment_type,segment_value
5714,a3e22242_526a_40ce_8483_5a9108bf3e0d,1013476,ДС,9,Порода,Песчаник глинистый
3388,242629b1_7f4b_42aa_bd86_774b5fa0264a,1004732,ДС,2,Порода,Песчаник глинистый
746,d438eeb5_2fbc_4501_beee_a7ce24a676c4,1000026,ДС,1,Порода,Песчаник


# Вспомогательные функции для изъятия файлов

## Функции для отображения и изъятия данных с диска

In [7]:
def show_image(image, nrows=1, ncols=1, cmap='gray'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 14))
    ax.imshow(image)
    ax.axis('off')
    return fig, ax

def get_photo(data, i):
    return io.imread(SAMPLE_CORE_PATH + '/photos/{}.jpeg'.format(data['photo_id'].iloc[i]))

def get_mask(data, i):
    return np.load(SAMPLE_CORE_PATH + '/matrixes/matrix_{}__{}.npz'.format(data['photo_id'].iloc[i], data['task_id'].iloc[i]))['data']

def get_json(data, i):
    datastore = ''
    with open(SAMPLE_CORE_PATH + '/extra/extra_{}__{}.json'.format(data['photo_id'].iloc[i],
                                                                   data['task_id'].iloc[i]), 'r', encoding="utf8") as f:
        datastore = json.load(f)

    return datastore

def show_all(data, i):
    print(data.iloc[i])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 14))
    ax1.imshow(get_photo(data, i))
    ax1.axis('off')

    cmap = plt.get_cmap('tab20b', np.max(get_mask(data, i)) + 1)
    mask = ax2.imshow(get_mask(data, i), cmap=cmap,
                      vmin=-0.5, vmax=np.max(get_mask(data, i))+0.5)
    ax2.axis('off')
    fig.colorbar(mask, ticks=np.arange(0, np.max(get_mask(data, i))+1))
    return fig, (ax1, ax2)



def show_photo_seies(data, column, value):
    fig, ax_ar = plt.subplots(1, 5, figsize=(14, 14))
    filt_data = data[data[column] == value]
    i = np.random.randint(0, filt_data.shape[0])
    for ax in ax_ar:
        filt_by_photo = filt_data[filt_data['photo_id'] == filt_data['photo_id'].iloc[i]]
        segments = filt_by_photo['segment_num'].values
        photo = get_photo(filt_data, i)
        mask = get_mask(filt_data, i)
        for value in segments:
            mask = np.ma.masked_where(mask == value, mask)
        
        ax.imshow(photo)
        ax.imshow(mask)
        ax.axis('off')
        filt_data.drop(filt_by_photo.index.values, axis=0, inplace=True)
        i = np.random.randint(0, filt_data.shape[0])
    
    return fig, ax_ar

## Функция для распаковки признаков, хранящихся в массивах

In [8]:
# unpack a column with arrays of features
def unpack_col(data, column):
    # count number of features in one array
    count = len(data[column].iloc[0])
    # initialize a column pattern
    col_p = column + '_{}'
    # initialize columns for the features
    for i in range(count):
        data[col_p.format(i)] = ""
    # unpacking
    for i in range(count):
        for j in range(len(data)):
            data[col_p.format(i)].iloc[j] = data[column].iloc[j][i]
    # convert every column to float32
    for i in range(count):
        data[col_p.format(i)] = data[col_p.format(i)].astype('float32')
    data.drop([column], axis=1, inplace=True)

## Функция для оценки эффективности данных. Использует RandomForest. 

In [9]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def make_experiment(X, y, random_states=(42, 14, 5), n_splits = 4):
    average_acc_score = 0.0
    average_f1_score = 0.0

    best_f1_score = 0.0
    worst_f1_score = 1.0
    for random_state in random_states:
        kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
        for train_index, test_index in kf.split(X):
            X_train, y_train = X[train_index],  y[train_index]
            X_test, y_test = X[test_index], y[test_index]

            model = RandomForestClassifier(n_estimators=750, bootstrap = True,
                                        random_state=random_state)
            model.fit(X_train, y_train)
            y_hat = model.predict(X_test)

            average_acc_score += metrics.accuracy_score(y_test, y_hat)
            
            f1_score = metrics.f1_score(y_test, y_hat, average='micro')
            if f1_score > best_f1_score:
                best_f1_score = f1_score
            if f1_score < worst_f1_score:
                worst_f1_score = f1_score
            average_f1_score += f1_score
    
    res = {}
    res['average Accuracy'] =  average_acc_score/n_splits/len(random_states)
    res['average F1'] = average_f1_score/n_splits/len(random_states)
    res['best F1'] = best_f1_score
    res['worst F1'] = worst_f1_score
    return res

# Работа с гистограммами изображений

## Подготовка функций для создания гистограмм для каждого сегмента

In [ ]:
from skimage.color import rgb2gray
from skimage.color import rgb2hsv
from skimage.exposure import equalize_hist

# Создаёт гистограмму серого изображения
def add_hist_rgb2gray(df, bins=32, equalize=False):
    data = df.copy()
    data['hist_gray'] = ""
    for i in range(data.shape[0]):
        # Taking a segment num
        segment_num = data['segment_num'].iloc[i]
        # Taking an image with the segment
        image = rgb2gray(get_photo(data, i))
        # Perform histogram equalization
        if equalize:
            image = equalize_hist(image)
        # Taking a part of the image by the segment num 
        remain_part = np.ma.masked_where(get_mask(data, i) == segment_num, image)
        # Calculate histogram over the image (all pixels outside segment are black)
        hist = np.histogram(remain_part.mask*remain_part.data, bins=bins, range=(0, 1))
        # Save histrogram without first bin
        data['hist_gray'].iloc[i] = hist[0][1:]

        if i % 20 == 0:
            print(i)
    print('rgb2gray hist complete')
    clear_output(wait=True)
    return data

# Создаёт RGB гистограмму
def add_hist_rgb(df, bins=32):
    data = df.copy()
    data['hist_rgb_0'] = ""
    data['hist_rgb_1'] = ""
    data['hist_rgb_2'] = ""
    for i in range(data.shape[0]):
        # Taking a segment num
        segment_num = data['segment_num'].iloc[i]
        # Taking an image with the segment
        image = get_photo(data, i)
        # For each channel...
        for j in range(3):
            # Taking a part of the image channel by the segment num 
            remain_part = np.ma.masked_where(get_mask(data, i) == segment_num, image[:,:,j])
            # Calculate histogram over the image channel (all pixels outside segment are black)
            hist = np.histogram(remain_part.mask*remain_part.data, bins=bins, range=(0, 1))
            # Save histrogram without first bin
            data['hist_rgb_{}'.format(j)].iloc[i] = hist[0][1:]

        if i % 20 == 0:
            print(i)
    print('hist_rgb hist complete')
    clear_output(wait=True)
    return data

# Создаёт HSV гистограмму
def add_hist_hsv(df, bins=32, equalize=False):
    data = df.copy()
    data['hist_hsv_0'] = ""
    data['hist_hsv_1'] = ""
    data['hist_hsv_2'] = ""
    for i in range(data.shape[0]):
        # Taking a segment num
        segment_num = data['segment_num'].iloc[i]
        # Taking an image with the segment
        image = rgb2hsv(get_photo(data, i))
        # Perform histogram equalization
        if equalize:
            image[:, :, 2] = equalize_hist(image[:, :, 2])
        # For each channel...
        for j in range(3):
            # Taking a part of the image channel by the segment num 
            remain_part = np.ma.masked_where(get_mask(data, i) == segment_num, image[:,:,j])
            # Calculate histogram over the image channel (all pixels outside segment are black)
            hist = np.histogram(remain_part.mask*remain_part.data, bins=bins, range=(0, 1))
            # Save histrogram without first bin
            data['hist_hsv_{}'.format(j)].iloc[i] = hist[0][1:]

        if i % 20 == 0:
            print(i)
    print('hist_hsv hist complete')
    clear_output(wait=True)
    return data

## Проверка гистограмм разного типа

### Гистограмма серого

In [ ]:
def experiment_gray(data_xy, bins, equalize=False, random_states = (42, 5, 14)):
    # calculate hist
    data_gray = add_hist_rgb2gray(data_xy, bins=bins, equalize=equalize)
    # extract a target value
    y_symb = data_gray['segment_value']
    # convert a target value to a numeric type
    y = pd.Series((pd.factorize(y_symb)[0]))
    # prepare an input data
    X = data_gray.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
    unpack_col(X, 'hist_gray')
    # make an experiment
    res = make_experiment(X.to_numpy(), y.to_numpy(), random_states, 4)
    return res

In [ ]:
def experiments_series_gray(data_xy, bins_range = (8, 256, 2)):
    data = pd.DataFrame(columns=('histogram', 'bins', 'equalize',
                                 'average Accuracy', 'average F1',
                                 'best F1', 'worst F1'))
    i = copy.copy(bins_range[0])
    index = 0
    is_equalize = 0
    while i <= bins_range[1]:
        res = experiment_gray(data_xy, i, is_equalize)
        res['histogram'] = 'gray'
        res['bins'] = i
        res['equalize'] = is_equalize
        data = data.append(res, ignore_index=True)
        print(data)
        i *= bins_range[2]

        if is_equalize == 0 and bins_range[1] < i:
            i = bins_range[0]
            is_equalize = 1
    clear_output(wait=True)
    return data

In [ ]:
data_exp_gray = experiments_series_gray(data_xy)
data_exp_gray.head(len(data_exp_gray))

,histogram,bins,equalize,average Accuracy,average F1,best F1,worst F1
0,gray,8,0,0.525000,0.525000,0.60,0.38
1,gray,16,0,0.538333,0.538333,0.64,0.44
2,gray,32,0,0.543333,0.543333,0.66,0.46
3,gray,64,0,0.568333,0.568333,0.64,0.50
4,gray,128,0,0.571667,0.571667,0.68,0.52
5,gray,256,0,0.568333,0.568333,0.68,0.48
6,gray,8,1,0.303333,0.303333,0.38,0.24
7,gray,16,1,0.361667,0.361667,0.44,0.28
8,gray,32,1,0.363333,0.363333,0.48,0.28
9,gray,64,1,0.413333,0.413333,0.54,0.30


В случае с гистограммой Серого, лучший результат был показан с 256 бинами без эквализации. Эквализация только портила полезную информацию.

### Гистограмма RGB

In [ ]:
def experiment_rgb(data_xy, bins, random_states = (42, 5, 14)):
    # calculate hist
    data_rgb = add_hist_rgb(data_xy, bins=bins)
    # extract a target value
    y_symb = data_rgb['segment_value']
    # convert a target value to a numeric type
    y = pd.Series((pd.factorize(y_symb)[0]))
    # prepare an input data
    X = data_rgb.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
    unpack_col(X, 'hist_rgb_0')
    unpack_col(X, 'hist_rgb_1')
    unpack_col(X, 'hist_rgb_2')
    # make an experiment
    res = make_experiment(X.to_numpy(), y.to_numpy(), random_states, 4)
    return res

In [ ]:
def experiments_series_rgb(data_xy, bins_range = (8, 256, 2)):
    data = pd.DataFrame(columns=('histogram', 'bins', 'equalize',
                                 'average Accuracy', 'average F1',
                                 'best F1', 'worst F1'))
    i = copy.copy(bins_range[0])
    index = 0
    is_equalize = 0
    while i <= bins_range[1]:
        res = experiment_rgb(data_xy, i)
        res['histogram'] = 'rgb'
        res['bins'] = i
        res['equalize'] = is_equalize
        data = data.append(res, ignore_index=True)
        print(data)
        i *= bins_range[2]

    clear_output(wait=True)
    return data

In [ ]:
data_exp_rgb = experiments_series_rgb(data_xy)
data_exp_rgb.head(len(data_exp_rgb))

,histogram,bins,equalize,average Accuracy,average F1,best F1,worst F1
0,rgb,8,0,0.333333,0.333333,0.40,0.26
1,rgb,16,0,0.335000,0.335000,0.40,0.26
2,rgb,32,0,0.331667,0.331667,0.40,0.26
3,rgb,64,0,0.330000,0.330000,0.40,0.26
4,rgb,128,0,0.330000,0.330000,0.38,0.24
5,rgb,256,0,0.331667,0.331667,0.40,0.26


Как можем видеть, гистограмма RGB показала себя очень плохо по сравнению с серой гистограммой. Для всех бинов вышел примерно одинаковый результат. 

### Гистограмма HSV

In [ ]:
def experiment_hsv(data_xy, bins, equalize=False, random_states = (42, 5, 14)):
    # calculate hist
    data_hsv = add_hist_hsv(data_xy, bins=bins, equalize=equalize)
    # extract a target value
    y_symb = data_hsv['segment_value']
    # convert a target value to a numeric type
    y = pd.Series((pd.factorize(y_symb)[0]))
    # prepare an input data
    X = data_hsv.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
    unpack_col(X, 'hist_hsv_0')
    unpack_col(X, 'hist_hsv_1')
    unpack_col(X, 'hist_hsv_2')
    # make an experiment
    res = make_experiment(X.to_numpy(), y.to_numpy(), random_states, 4)
    return res

In [ ]:
def experiments_series_hsv(data_xy, bins_range = (8, 256, 2)):
    data = pd.DataFrame(columns=('histogram', 'bins', 'equalize',
                                 'average Accuracy', 'average F1',
                                 'best F1', 'worst F1'))
    i = copy.copy(bins_range[0])
    index = 0
    is_equalize = 0
    while i <= bins_range[1]:
        res = experiment_hsv(data_xy, i, is_equalize)
        res['histogram'] = 'hsv'
        res['bins'] = i
        res['equalize'] = is_equalize
        data = data.append(res, ignore_index=True)
        print(data)
        i *= bins_range[2]

        if is_equalize == 0 and bins_range[1] < i:
            i = bins_range[0]
            is_equalize = 1
    clear_output(wait=True)
    return data

In [ ]:
data_exp_hsv = experiments_series_hsv(data_xy)
data_exp_hsv.head(len(data_exp_hsv))

,histogram,bins,equalize,average Accuracy,average F1,best F1,worst F1
0,hsv,8,0,0.611667,0.611667,0.78,0.48
1,hsv,16,0,0.631667,0.631667,0.80,0.54
2,hsv,32,0,0.630000,0.630000,0.80,0.52
3,hsv,64,0,0.655000,0.655000,0.78,0.56
4,hsv,128,0,0.650000,0.650000,0.78,0.56
5,hsv,256,0,0.641667,0.641667,0.74,0.52
6,hsv,8,1,0.510000,0.510000,0.62,0.44
7,hsv,16,1,0.530000,0.530000,0.64,0.46
8,hsv,32,1,0.566667,0.566667,0.72,0.48
9,hsv,64,1,0.590000,0.590000,0.74,0.50


Самый лучший результат, в случае с HSV гистограммой, показала гистограмма без эквализации с 256 бинами.

Эквализация гистограммы по серому изображению нигде не привела к хорошему результату.
HSV показал себя немного лучше Серого представления. Вероятно, это связано с тем, что серый канал несёт в себе основную информацию. При этом два остальных цветовых канала немного дополняют её. 

# Проверка текстурных оценок на основе GLCM 

Функция для создания текстурных оценок на основе GLCM

In [ ]:
from skimage.feature import greycomatrix, greycoprops
from skimage.util import img_as_ubyte
from enum import Enum

def add_GLCM_feature(df, bit_mask):
    # copy data
    data = df.copy()

    # initialize enum with features
    class Features(Enum):
        contrast = 0
        dissimilarity = 1
        homogeneity = 2
        ASM = 3
        energy = 4
        correlation = 5

    # initialize a column for features 
    data['glcm_features'] = ""

    # for each row in data...
    for i in range(data.shape[0]):
        # take a segment from a photo
        segment_num = data['segment_num'].iloc[i]
        remain_part = np.ma.masked_where(get_mask(data_xy, i) == segment_num, img_as_ubyte(rgb2gray(get_photo(data_xy, i))))
        remain_part.data[remain_part.data == 0] = 1
        remain_part.data[~remain_part.mask] = 0
        # calculate GLCM (don't take dark pixels, since they're damaged)
        glcm = greycomatrix(remain_part.data*remain_part.mask, distances=[1, 2, 3], angles=[0, 45, 90, 135], symmetric=True , normed=True)[1:, 1:, :, :]
        
        # initialized list for features
        l = []

        # for each feature in enum...
        for feature in Features:
            # check if a feature is needed..
            if bit_mask[feature.value]:
                # calculate a feature and add it to list
                [l.append(item) for item in greycoprops(glcm, feature.name).reshape(-1)]

        # add feature list to the column
        data['glcm_features'].iloc[i] = l

        if i % 20 == 0:
            print(i)
    print('adding GLCM features is complete')
    clear_output(wait=True)
    return data

In [ ]:
def experiment_glcm(df, random_states=(42, 14, 5)):
    data = pd.DataFrame(columns=('bitmask', 'average Accuracy', 'average F1',
                                'best F1', 'worst F1'))

    for i in range(1, 64):
        bit_mask = list(map(int, list('{0:06b}'.format(i))))
        # calculate GLCM features
        data_glcm = add_GLCM_feature(df, bit_mask)
        # extract a target value
        y_symb = data_glcm['segment_value']
        # convert a target value to a numeric type
        y = pd.Series((pd.factorize(y_symb)[0]))
        # prepare an input data
        X = data_glcm.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
        unpack_col(X, 'glcm_features')
        # make an experiment
        exp_res = make_experiment(X.to_numpy(), y.to_numpy(), random_states, 4)
        
        exp_res['bitmask'] = bit_mask
        data = data.append(exp_res, ignore_index=True)
        print(data)
    clear_output(wait=True)
    return data
    

Проведём эксперимент для каждой комбинации дескрипторов. Всего 64.

In [ ]:
data_exp_glcm = experiment_glcm(data_xy)
data_exp_glcm.head(10)

,bitmask,average Accuracy,average F1,best F1,worst F1
0,"[0, 0, 0, 0, 0, 1]",0.366667,0.366667,0.42,0.28
1,"[0, 0, 0, 0, 1, 0]",0.390000,0.390000,0.50,0.32
2,"[0, 0, 0, 0, 1, 1]",0.390000,0.390000,0.50,0.30
3,"[0, 0, 0, 1, 0, 0]",0.390000,0.390000,0.50,0.32
4,"[0, 0, 0, 1, 0, 1]",0.388333,0.388333,0.50,0.30
5,"[0, 0, 0, 1, 1, 0]",0.393333,0.393333,0.52,0.32
6,"[0, 0, 0, 1, 1, 1]",0.393333,0.393333,0.52,0.32
7,"[0, 0, 1, 0, 0, 0]",0.473333,0.473333,0.56,0.36
8,"[0, 0, 1, 0, 0, 1]",0.463333,0.463333,0.52,0.38
9,"[0, 0, 1, 0, 1, 0]",0.476667,0.476667,0.60,0.38


Выведем 10 лучших результатов, отсортированных по average F1 и best F1.

In [ ]:
data_exp_glcm.sort_values(by=['average F1', 'best F1'], ascending=False).head(10)

,bitmask,average Accuracy,average F1,best F1,worst F1
55,"[1, 1, 1, 0, 0, 0]",0.546667,0.546667,0.68,0.38
48,"[1, 1, 0, 0, 0, 1]",0.523333,0.523333,0.64,0.42
59,"[1, 1, 1, 1, 0, 0]",0.521667,0.521667,0.60,0.38
57,"[1, 1, 1, 0, 1, 0]",0.520000,0.520000,0.60,0.38
39,"[1, 0, 1, 0, 0, 0]",0.520000,0.520000,0.60,0.36
50,"[1, 1, 0, 0, 1, 1]",0.515000,0.515000,0.60,0.38
52,"[1, 1, 0, 1, 0, 1]",0.513333,0.513333,0.60,0.38
56,"[1, 1, 1, 0, 0, 1]",0.511667,0.511667,0.60,0.38
23,"[0, 1, 1, 0, 0, 0]",0.510000,0.510000,0.64,0.32
16,"[0, 1, 0, 0, 0, 1]",0.510000,0.510000,0.64,0.40


Как видим, модель демонстрирует лучшие результаты при использовании Contrast, Dissimilarity и Homogeneity. Но как мы можем помнить из описания их работы, они сильно коррелируют между собой. Вероятно, из-за особенности структуры RandomForest и деревьев решений в частности, получилось так, что не смотря на измерение сильно кореллирубщих между собой сущностей, вместе они улучшают работу друг-друга по-отдельности. Возможно, их небольшие отличия между собой вносят корректировки в модель и пораждают более точный результат.

# Когерентные цветовые вектора

## Реализая CCV: https://github.com/tamanobi/ccv

In [ ]:
import sys
from skimage.util import img_as_ubyte
import cv2

def QuantizeColor(img, n=64):
  div = 256//n
  rgb = cv2.split(img)
  q = []
  for ch in rgb:
    vf = np.vectorize(lambda x, div: int(x//div)*div)
    quantized = vf(ch, div)
    q.append(quantized.astype(np.uint8))
  d_img = cv2.merge(q)
  return d_img

"""
Proccess of Computing CCV(color coherence vector)
@see http://vis.uky.edu/~cheung/courses/ee639_fall04/readings/ccv.pdf 
1. Blur
2. Quantizing color
3. Thresholding
4. Labeling
5. Counting
"""
def ccv(src, tau=0, n=64):
  img = src.copy()
  row, col, channels = img.shape
  if not col == 300:
    aspect = 300.0//col
    try:
        img = cv2.resize(img, None, fx=aspect, fy=aspect, interpolation = cv2.INTER_CUBIC)
        2 + 2
    except:
        2 + 2
  row, col, channels = img.shape
  # blur
  img = cv2.GaussianBlur(img, (3,3),0)
  # quantize color
  img = QuantizeColor(img, n)
  bgr = cv2.split(img)
  #bgr = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
  if tau == 0:
    tau = row*col * 0.1
  alpha = np.zeros(n)
  beta = np.zeros(n)
  # labeling
  for i,ch in enumerate(bgr):
    ret,th = cv2.threshold(ch,127,255,0)
    ret, labeled, stat, centroids = cv2.connectedComponentsWithStats(th, None, cv2.CC_STAT_AREA, None, connectivity=8)
    #!see https://github.com/atinfinity/lab/wiki/OpenCV%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E3%83%A9%E3%83%99%E3%83%AA%E3%83%B3%E3%82%B0#samplecode
    #!see http://docs.opencv.org/3.0.0/d3/dc0/group__imgproc__shape.html#gac7099124c0390051c6970a987e7dc5c5
    # generate ccv
    areas = [[v[4],label_idx] for label_idx,v in enumerate(stat)]
    coord = [[v[0],v[1]] for label_idx,v in enumerate(stat)]
    # Counting
    for a,c in zip(areas,coord):
      area_size = a[0]
      x,y = c[0], c[1]
      if (x < ch.shape[1]) and (y < ch.shape[0]):
        bin_idx = int(ch[y,x]//(256//n))
        if area_size >= tau:
          alpha[bin_idx] = alpha[bin_idx] + area_size
        else:
          beta[bin_idx] = beta[bin_idx] + area_size
  return alpha, beta

def ccv_plot(img, alpha, beta, n=64):
  import matplotlib.pyplot as plt
  X = [x for x in range(n*2)]
  Y = alpha.tolist()+beta.tolist()
  with open('ccv.csv','w') as f:
    f.write(str(Y))
  im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.subplot(2,1,1)
  plt.imshow(im)
  plt.subplot(2,1,2)
  plt.bar(X, Y, align='center')
  #plt.yscale('log')
  #plt.xticks(X, (['alpha']*n)+(['beta']*n))
  plt.show()  

## Функция для добавления CCV дескрипторов в датасет

In [ ]:
def add_CCV_feature(df, bins=16, alpha_on=True, betta_on=True):
    # copy data
    data = df.copy()

    # initialize a column for features 
    data['ccv_features'] = ""

    # for each row in data...
    for i in range(data.shape[0]):
        # take a segment from a photo
        segment_num = data['segment_num'].iloc[i]
        
        image = img_as_ubyte(get_photo(data, i))
        for j in range(3):
            remain_part = np.ma.masked_where(get_mask(data, i) == segment_num, image[:, :, j])
            image[:,:, j] = remain_part.data*remain_part.mask

        # calculate CCV
        alpha, betta = ccv(image, n=bins)
        # alpha = list(map(bool, alpha))
        # betta = list(map(bool, betta)) 

        # initialized list for features
        l = []

        if alpha_on:
            [l.append(item) for item in alpha[1:]]
        
        if betta_on:
            [l.append(item) for item in betta[1:]]

        # add feature list to the column
        data['ccv_features'].iloc[i] = l

        if i % 20 == 0:
            print(i)
    print('adding CCV features is complete')
    clear_output(wait=True)
    return data

## Функция для проведения эксперимента по CCV векторам

In [ ]:
def experiment_ccv(data_xy, bins, alpha_on=True, betta_on=True, random_states = (42, 5, 14)):
    # calculate hist
    data_ccv = add_CCV_feature(data_xy, bins=bins, alpha_on=alpha_on, betta_on=betta_on)
    # extract a target value
    y_symb = data_ccv['segment_value']
    # convert a target value to a numeric type
    y = pd.Series((pd.factorize(y_symb)[0]))
    # prepare an input data
    X = data_ccv.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
    unpack_col(X, 'ccv_features')
    # make an experiment
    res = make_experiment(X.to_numpy(), y.to_numpy(), random_states, 4)
    return res

## Функция для проведения серии экспериментов по CCV векторам

In [ ]:
def experiments_series_ccv(data_xy, bins_range = (8, 256, 2)):
    data = pd.DataFrame(columns=('bins',
                                 'average Accuracy', 'average F1',
                                 'best F1', 'worst F1'))
    i = copy.copy(bins_range[0])
    index = 0
    while i <= bins_range[1]:
        res = experiment_ccv(data_xy, i)
        res['bins'] = i
        data = data.append(res, ignore_index=True)
        print(data)
        i *= bins_range[2]

    clear_output(wait=True)
    return data

Проведём эксперименты для CCV дескрипторов с разным количеством бинов.

In [ ]:
data_exp_ccv = experiments_series_ccv(data_xy)
data_exp_ccv.head(len(data_exp_ccv))

,bins,average Accuracy,average F1,best F1,worst F1
0,8.0,0.420000,0.420000,0.48,0.36
1,16.0,0.411667,0.411667,0.46,0.32
2,32.0,0.413333,0.413333,0.52,0.34
3,64.0,0.408333,0.408333,0.50,0.34
4,128.0,0.426667,0.426667,0.48,0.38
5,256.0,0.423333,0.423333,0.52,0.34


Как видим, результаты не сильно отличаются при разной степени дискретизации цветового пространства. Однако, всё же лучший численный результат показал CCV дескриптор с 128 бинами. 

# Финальная модель

Расчитаем теперь результаты модели с дескрипторами из каждой категории, которые показали себя лучше всего. 

In [ ]:
data_final = add_hist_hsv(data_xy, bins=128, equalize=False)
data_final = add_GLCM_feature(data_final, [1, 1, 1, 0, 0, 0])
data_final = add_CCV_feature(data_final, bins=256)

y_symb = data_final['segment_value']
# convert a target value to a numeric type
y = pd.Series((pd.factorize(y_symb)[0]))
# prepare an input data
X = data_final.drop(['task_id', 'photo_id', 'photo_type', 'segment_num', 'segment_type', 'segment_value'], axis=1)
unpack_col(X, 'glcm_features')
unpack_col(X, 'ccv_features')
unpack_col(X, 'hist_hsv_0')
unpack_col(X, 'hist_hsv_1')
unpack_col(X, 'hist_hsv_2')
# make an experiment
exp_res = make_experiment(X.to_numpy(), y.to_numpy(), (42, 14, 5), 4)
exp_res

{'average Accuracy': 0.6883333333333334,
 'average F1': 0.6883333333333334,
 'best F1': 0.8399999999999999,
 'worst F1': 0.6}